**`Webinar 2024-02-21`**

[YouTube Recording](https://www.youtube.com/watch?v=TSbiawPaQcQ&list=PLePKvgYhNOFxg3O7gXfmxaKp0EWXEhvNC&index=11)

# Book Ideation and Generation using Document Review and Long Form

### Overview

The goal is to write a quality educational book in the simplest way possible. We will go through the ideation phase to set the tone and topics of the book.

##### Project Flow

1. Create knowledge bases for each chapter we want in the book.
2. Get some user inputs in regard to overall theme and style.
3. For each chapter knowledge base, run a document review where the "final review" output is a chapter outline.
4. Feed each of the document review outlines into a long form generation call.

##### Initialization

##### `NOTE: To run this, you will need to replace with your own chapter KBs as well as book themes/guidance in the next 2 cells`

In [1]:
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = ''
SP_API_KEY_SECRET = ''
API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)
superpowered.set_api_key(SP_API_KEY_ID, SP_API_KEY_SECRET)

knowledge_base_ids = [
    '08844d05-4e09-479d-9770-6faebccfceb6',  # STOICISM
    'a0fd538c-06ec-43ae-849b-f9570caca2c6',  # EPICUREANISM
    '87401ada-dfcb-4e3a-afbc-8fe9ead62bb4',  # MINIMALISM
    '6fde0338-0856-477b-bfa0-439432dfbcd1',  # HEDONISM
    'a78019b4-c498-412b-9daf-737698fab4c5',  # EXISTENTIALISM
    '740b2668-ecab-4372-9bc4-6a01a4f88560',  # NIHILISM
    '183cee2f-2053-480b-8e86-9377489208c5',  # PRAGMATISM
    '0e75e305-b314-40a3-a4fb-7f357bda0c10',  # IDEALISM
]


##### User input describing goal for the book

In [2]:
BOOK_DIRECTORY = 'living_a_good_life'
BOOK_TITLE = 'Living a Good Life in the Age of AI and Automation'
BOOK_THEME = 'The general theme of this book is to look at what it means to live a good life in the context of stoicism, epicureanism, minimalism, hedonism, existentialism, nihilism, pragmatism, and idealism. Each chapter will cover one of these 8 philosophical schools of thought.'
BOOK_OVERALL_STYLE_GUIDANCE = 'The style should be widely accessible to the general public with a focus on how each of the philosophies are relevant in the modern age of AI and automation. It is important to use a variable vocabulary and make sure that nothing is repetitive and dry. Be fun and present ideas with examples. Avoid bland closing statements like "in summary", "in conclusion", etc. The book should be a fun and engaging read.'
BOOK_CHAPTER_STYLE_GUIDANCE = 'Each chapter should start with a general overview of the philosophy in question and explain how the concepts are relevant in the modern age of AI and automation. Give examples of what living by this philosophy would look like in scenarios like: "remote work and lack of social interacation", "complete automation with no jobs", "human population expantion", "virtual reality", "augmented reality", "cell phones", "immersive video games", etc.'

##### Helper functions that will run document reviews and long form plus stream results

- `run_and_display_review()` function will start a document review, show an active reading progress bar and then stream final review results as they happen.
- `submit_long_form_job()` just submits a long form job so we can generate all the chapters in parallel.


In [3]:
def run_and_display_review(kb: dict, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb["id"]}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description(f'Active Reading Progress for {kb["title"]}')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']


def submit_long_form_job(kb: dict, prompt: str):
    url = f'{SP_BASE_URL}/long_form'
    payload = {
        'knowledge_base_ids': [kb['id']],
        'model': 'gpt-4',
        'response_length': 'long',
        'prompt': prompt,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    return resp.json()['id']


##### Loop through each knowledge base and run the document review

In [4]:
active_reading_instructions = f"""\
You are reading through a knowledge base on {{philosophy}}. Please extract information that is relevant to living a good life.

The infomration that is extracted will be used to write a chapter about {{philosophy}} for a book with the following theme: {BOOK_THEME}

Your output should be a list of key points and examples that could be relevant to the modern age of AI and automation. The information you are reading through might not directly mention AI and automation so it's up to you to decide if the information is relevant or not.
"""

final_review_instructions = f"""\
Given the list key points and examples from the active reading, please generate a chapter outline. The book must follow these style guidelines:

1. {BOOK_OVERALL_STYLE_GUIDANCE}
2. {BOOK_CHAPTER_STYLE_GUIDANCE}

The chapter outline should contain a list of detailed section descriptions. Please try to keep the number of sections between 5 and 7
"""

outlines = []
for kb_id in knowledge_base_ids:
    # get the knowledge base object
    kb_url = f'{SP_BASE_URL}/knowledge_bases/{kb_id}'
    resp = requests.get(kb_url, auth=API_AUTH)
    kb = resp.json()

    outline = run_and_display_review(kb, active_reading_instructions.format(philosophy=kb['title']), final_review_instructions)
    outlines.append((kb, outline))

### Chapter Outline: Stoicism and Modern Life in the Age of AI and Automation

#### Introduction to Stoicism
- Brief history of Stoicism, from its origins with Zeno of Citium in Athens to its development and influence in Rome.
- Overview of Stoic philosophy, focusing on virtue ethics, the concept of living in accordance with nature, and the Stoic path to happiness (eudaimonia).
- The relevance of Stoicism in the context of modern challenges such as AI, automation, and the digital age.

#### Stoicism and the Digital Age
- Exploration of how Stoic principles can guide individuals in navigating the complexities of remote work, social isolation, and the digital overload of information.
- Examples of applying Stoic practices to maintain emotional resilience and mental clarity amidst the constant presence of technology.

#### Virtue in an Automated World
- Discussion on the Stoic virtues of wisdom, courage, temperance, and justice in the context of a world where AI and automation play significant roles in decision-making and ethics.
- How Stoicism can inform ethical programming and the development of AI to serve the common good.

#### Stoicism and Environmental Sustainability
- Examination of Stoic views on nature and how these can inspire sustainable living and environmental responsibility in an era of material consumption and technological advancement.
- Stoic practices that promote minimalism and the appreciation of "indifferents" to combat the environmental impact of consumerism.

#### Emotional Resilience in Virtual Realities
- Analysis of Stoic emotional ethics, focusing on apatheia (freedom from passion) and the management of emotions in virtual and augmented realities.
- The role of Stoicism in developing a balanced approach to immersive technologies, ensuring they enhance rather than detract from the quality of life.

#### Stoicism and Community in the Age of Individualism
- The Stoic concept of cosmopolitanism and its implications for fostering a sense of global community and empathy in an increasingly interconnected yet fragmented world.
- Practical Stoic exercises to cultivate social virtues and a sense of belonging in both physical and virtual communities.

#### Living Stoically Amidst Uncertainty
- Strategies for applying Stoic wisdom to embrace uncertainty and change, particularly in the face of rapid technological advancements and the potential for societal upheaval.
- The Stoic practice of focusing on what is within one's control as a means of navigating the challenges and opportunities presented by AI and automation.

#### Conclusion: The Timeless Wisdom of Stoicism
- Reflection on the enduring relevance of Stoic philosophy in providing guidance for living a good life in the modern age.
- Encouragement for readers to explore Stoic practices in their daily lives, fostering resilience, virtue, and happiness amidst the challenges of the 21st century.

This chapter aims to engage readers with a lively and accessible exploration of Stoicism, illustrating its practical applications in the context of contemporary issues such as AI, automation, and the digital revolution. Through real-world examples and practical advice, the chapter will demonstrate how ancient Stoic wisdom can offer valuable insights for navigating the complexities of modern life.

Active Reading Progress for STOICISM [CH. 1]: 100%|██████████| 1/1 [02:16<00:00, 136.45s/it]      


### Chapter Outline: Epicureanism in the Age of AI and Automation

#### Introduction to Epicureanism
- Brief history of Epicureanism, founded by Epicurus in 307 BCE.
- Core principles: seeking modest, sustainable pleasure and freedom from fear.
- The relevance of Epicureanism in the context of modern challenges like AI, automation, and digital life.

#### Epicureanism vs. Modern Hedonism
- Clarification of common misconceptions: Epicureanism is not about indulgence in fleeting pleasures but about sustainable happiness.
- Comparison with modern hedonism, especially in the digital age (social media, virtual reality).
- Examples of how to apply Epicurean moderation in the consumption of digital content and technology.

#### The Epicurean Pursuit of Pleasure in an Automated World
- The impact of complete automation on jobs and the Epicurean response: finding pleasure in simplicity and intellectual pursuits.
- The role of AI in enhancing the Epicurean lifestyle, such as reducing fear and pain through medical advancements.
- The challenge of maintaining human connections in a world of remote work and virtual interactions.

#### Epicureanism and the Digital Social Sphere
- The importance of friendship in Epicureanism and its application in the age of social media and online communities.
- Navigating the balance between virtual friendships and the need for physical community.
- Strategies for using technology to enhance, rather than replace, meaningful human connections.

#### Epicurean Ethics in the Age of AI
- The application of Epicurean virtues (courage, moderation) in ethical dilemmas posed by AI and automation.
- The role of AI in justice and the Epicurean principle of not harming or being harmed.
- The potential for AI to fulfill Epicurean desires for knowledge and dispel fears, particularly about the unknown.

#### Living an Epicurean Life Amidst Technological Advancement
- Practical tips for applying Epicurean principles to daily life in a technologically advanced society.
- Balancing the use of technology for pleasure and utility without succumbing to excess.
- The importance of self-reflection and moderation in desires in an age of endless digital possibilities.

#### Conclusion: The Timeless Relevance of Epicureanism
- Recap of how Epicureanism provides a framework for living a good life amidst the challenges and opportunities of the modern age.
- The enduring appeal of Epicureanism in promoting a life of tranquility, friendship, and meaningful pleasure.
- Encouragement to explore Epicurean practices as a way to navigate the complexities of life in the digital era.

This chapter aims to engage readers with a blend of historical philosophy and contemporary application, making Epicureanism accessible and relevant to the challenges and opportunities presented by AI, automation, and the digital world. Through practical examples and a focus on sustainable pleasure and freedom from fear, readers will discover how ancient wisdom can guide modern living.

Active Reading Progress for EPICUREANISM [CH. 2]: 100%|██████████| 1/1 [01:27<00:00, 87.34s/it]      


### Chapter Outline: Embracing Minimalism in the Age of AI and Automation

#### Introduction to Minimalism
- Begin with a brief history of minimalism, touching on its origins in post-World War II art and its evolution into a broader lifestyle philosophy.
- Explain how minimalism, with its emphasis on simplicity, reduction, and focusing on essential elements, offers a counterbalance to the complexities introduced by AI and automation in modern life.

#### Minimalism and Modern Technology
- Explore the relationship between minimalism and technology, particularly in the context of AI, virtual reality, augmented reality, and immersive video games.
- Discuss how adopting a minimalist approach can help manage digital clutter, reduce tech-induced stress, and promote a healthier engagement with technology.

#### Minimalism in the Digital Workplace
- Address the challenges and opportunities of remote work, including the lack of social interaction and the potential for complete automation of jobs.
- Offer insights into how minimalism can guide individuals in creating a balanced and productive remote work environment, focusing on essential tasks and minimizing distractions.

#### Minimalism and Environmental Sustainability
- Examine the role of minimalism in promoting environmental sustainability, especially in the face of human population expansion and climate change.
- Highlight examples of minimalist practices, such as capsule wardrobes and minimalist architecture, that contribute to reducing waste and conserving resources.

#### Minimalism in Personal Finance and Consumption
- Discuss the concept of financial minimalism and its relevance in an era of consumerism exacerbated by targeted advertising and e-commerce platforms powered by AI.
- Provide strategies for applying minimalism to personal finance and consumption, aiming for a more intentional and less stressful lifestyle.

#### Minimalism as a Tool for Well-being
- Delve into the psychological and emotional benefits of minimalism, including reduced stress, increased focus, and greater appreciation for simple pleasures.
- Share practical tips for incorporating minimalism into various aspects of life, from decluttering physical spaces to simplifying schedules and relationships.

#### The Future of Minimalism
- Speculate on the future trajectory of minimalism in a world increasingly shaped by AI, automation, and digital innovation.
- Encourage readers to consider how minimalism can serve as a guiding philosophy for navigating the complexities of modern life, ensuring that technology serves humanity rather than overwhelms it.

Throughout the chapter, use engaging examples and anecdotes to illustrate key points, making the content relatable and actionable for readers. Avoid technical jargon and aim for a tone that is both informative and inspiring, inviting readers to explore how minimalism can enhance their lives in the age of AI and automation.

...

Active Reading Progress for NIHILISM [CH.6]: 100%|██████████| 1/1 [01:37<00:00, 97.32s/it]       


**Chapter Outline: Pragmatism and the Good Life in the Age of AI and Automation**

**Introduction to Pragmatism**
- Begin with a brief history of pragmatism, highlighting its origins in the United States in the late 19th century with key figures such as Charles Sanders Peirce, William James, and John Dewey.
- Explain pragmatism's core principle: the truth of an idea or theory is determined by its practical consequences and utility in solving problems.
- Introduce the relevance of pragmatism in the context of modern challenges such as AI, automation, and digital transformation.

**Pragmatism in a World of AI and Automation**
- Discuss how pragmatism's focus on practical consequences and problem-solving is particularly relevant in a world increasingly dominated by AI and automation.
- Explore scenarios such as remote work, social isolation, job displacement due to automation, and the ethical implications of AI decisions.
- Use examples to illustrate how a pragmatic approach can guide ethical AI development, emphasizing the importance of outcomes and the adaptability of ethical standards.

**Pragmatism and Digital Well-being**
- Examine the impact of digital technologies like smartphones, virtual reality, and immersive video games on individual well-being and social interactions.
- Discuss how pragmatism advocates for a balanced approach to technology, focusing on its practical benefits while being mindful of potential negative consequences.
- Highlight pragmatic strategies for enhancing digital well-being, such as promoting meaningful online interactions and using technology to support rather than replace real-world experiences.

**Pragmatism and Environmental Challenges**
- Address the pragmatic approach to environmental issues, emphasizing the importance of practical solutions that balance human needs with ecological sustainability.
- Discuss how pragmatism can inform policies on climate change, resource management, and technological innovation in green technologies.
- Provide examples of pragmatic environmental initiatives that have successfully balanced economic development with environmental protection.

**Pragmatism in Education and Lifelong Learning**
- Explore the implications of pragmatism for education, particularly in the context of lifelong learning and the need for skills adaptation in a rapidly changing world.
- Discuss John Dewey's contributions to pragmatic pedagogy, emphasizing the importance of experiential learning and critical thinking.
- Highlight the role of education in preparing individuals for the challenges of a digital economy, including the need for digital literacy and adaptability.

**Pragmatism, Ethics, and Social Responsibility**
- Delve into the ethical dimensions of pragmatism, particularly its emphasis on the practical consequences of actions and decisions.
- Discuss how pragmatism can inform ethical decision-making in business, technology, and public policy, focusing on outcomes and the common good.
- Explore the role of pragmatism in promoting social responsibility, democratic participation, and the pursuit of justice in an increasingly complex world.

**Conclusion: Living the Good Life with Pragmatism**
- Summarize the key points of the chapter, emphasizing the relevance of pragmatism in addressing contemporary challenges.
- Highlight the flexibility and adaptability of pragmatism as a philosophical approach that encourages continuous inquiry, experimentation, and revision of beliefs based on practical outcomes.
- Conclude with a reflection on how pragmatism offers a framework for living a good life in the modern age, balancing the pursuit of individual fulfillment with social responsibility and ethical considerations.

This chapter outline aims to engage readers with a dynamic and accessible exploration of pragmatism, illustrating its relevance and application in various aspects of modern life, from technology and ethics to education and environmental sustainability.

Active Reading Progress for PRAGMATISM [CH. 7]: 100%|██████████| 1/1 [01:57<00:00, 117.97s/it]      


**Chapter Outline: Exploring Idealism in the Age of AI and Automation**

**Introduction to Idealism: A Mind-Centric Universe**
- Begin with a captivating introduction to idealism, emphasizing its core assertion that reality is fundamentally mental or consciousness-based. 
- Present a brief historical overview, highlighting contributions from key philosophers like Berkeley, Kant, Hegel, and Fichte.
- Transition into the relevance of idealism in the context of AI and automation, setting the stage for a deep dive into how these concepts intersect with modern technological advancements.

**Section 1: The Idealist Perspective on AI and Consciousness**
- Explore the implications of idealism for understanding artificial intelligence, particularly the quest for AI consciousness.
- Discuss the idea that if reality is fundamentally mental, then AI, as part of this reality, might also possess or simulate aspects of consciousness.
- Use examples from AI research that blurs the line between computational processes and cognitive functions, reflecting on idealist interpretations.

**Section 2: Automation and the Idealist View of Work**
- Delve into the idealist perspective on the shift towards complete automation and the diminishing role of human labor.
- Consider how idealism values consciousness and mental activity over physical labor, suggesting a reevaluation of what constitutes "work" in an automated world.
- Discuss the potential for a society where creative and intellectual pursuits become the primary form of human contribution, inspired by idealist principles.

**Section 3: Virtual Realities and the Idealist Notion of Experience**
- Examine the intersection of idealism with virtual and augmented reality technologies, considering the idea that these experiences are as "real" as physical ones if reality is consciousness-based.
- Discuss the implications for identity, social interaction, and the nature of experience itself when life can be lived in entirely constructed realities.
- Highlight examples from VR and AR that challenge our traditional notions of space, time, and presence.

**Section 4: The Digital Mind: Social Media, Smartphones, and Consciousness**
- Investigate how idealism can inform our understanding of the pervasive influence of digital technologies like social media and smartphones on our consciousness and perception of reality.
- Discuss the idea that these technologies extend the mind and create shared mental spaces, reflecting on the implications for individuality and collective consciousness.
- Consider the role of algorithms in shaping our mental landscape and the idealist notion that reality is a construct of the mind.

**Section 5: The Ethical Implications of Idealism in a Technologically Advanced World**
- Explore the ethical considerations that arise from an idealist perspective in the age of AI and automation.
- Discuss how idealism's emphasis on the primacy of consciousness could inform ethical AI development, digital privacy, and the rights of potentially conscious AI entities.
- Reflect on the moral responsibilities of creators and users of technology in a world where the mental and the digital are increasingly intertwined.

**Conclusion: Embracing an Idealist Future**
- Conclude with a thought-provoking reflection on the potential for a future society shaped by idealist principles, where consciousness and mental creativity are at the forefront of human endeavor.
- Encourage readers to consider the implications of idealism for their own lives and the collective future, emphasizing the transformative potential of viewing reality through an idealist lens.
- Avoid cliché summaries, instead leaving readers with questions and curiosities about the nature of reality, consciousness, and the role of technology in an idealist world.

This chapter aims to engage readers with a dynamic and accessible exploration of idealism, making connections to contemporary issues and technologies while encouraging deep reflection on the nature of reality and consciousness.

Active Reading Progress for IDEALISM [CH. 8]: 100%|██████████| 1/1 [03:33<00:00, 213.30s/it]      


##### Create the prompt for each chapter with a little semantic sugar around the outlines created by the document review step and submit long form job

In [5]:
chapter_ids = []

for kb, outline in outlines:
    prompt = f"""\
    You have been assigned to write a chapter in a book titled "{BOOK_TITLE}" about {kb['title']}. Please follow this outline when writing the chapter:

    {outline}

    {BOOK_OVERALL_STYLE_GUIDANCE}
    {BOOK_CHAPTER_STYLE_GUIDANCE}

    DO NOT EVER USE CLOSINGS LIKE "in conclusion", "in summary", "ultimately", etc. They are boring and you should have more pride in your output.

    When you generate a title, please include the chapter number and the name of the philosophy. This chapter is {kb['title']}".
    """
    print('STARTING CHAPTER:', kb['title'])
    chapter_job_id = submit_long_form_job(kb, prompt)
    chapter_ids.append(chapter_job_id)
    

STARTING CHAPTER: STOICISM [CH. 1]
STARTING CHAPTER: EPICUREANISM [CH. 2]
STARTING CHAPTER: MINIMALISM [CH. 3]
STARTING CHAPTER: HEDONISM [CH. 4]
STARTING CHAPTER: EXISTENTIALISM [CH. 5]
STARTING CHAPTER: NIHILISM [CH.6]
STARTING CHAPTER: PRAGMATISM [CH. 7]
STARTING CHAPTER: IDEALISM [CH. 8]


# START OF PART 2 --- 2024-02-21

## Finishing touches

- Edit the title and export each chapter as a markdown file and put in the book folder.
- Generate an "Introduction" and "Conclusion" chapter by running a document review on a knowledge base with all the outlines and generated chapters. These reviews will gnerate the outlines and then be fed into the `long_form` endpoint.
- Use pandoc to export the .epub file.

##### Edit titles so they are simply the name of the philosophy

In [10]:
def patch_chapter_title(new_title, job_id):
    url = f'{SP_BASE_URL}/long_form/{job_id}'
    resp = requests.patch(url, json={'title': new_title}, auth=API_AUTH)
    return resp.json()

for (kb, _), chapter_job_id in zip(outlines, chapter_ids):
    # extract philosophy in question from the KB title and make that the chapter title
    philosophy = kb['title'].split(' ')[0].capitalize()
    print(f'PATCHING CHAPTER: {philosophy}')
    resp = patch_chapter_title(philosophy, chapter_job_id)


PATCHING CHAPTER: Stoicism
PATCHING CHAPTER: Epicureanism
PATCHING CHAPTER: Minimalism
PATCHING CHAPTER: Hedonism
PATCHING CHAPTER: Existentialism
PATCHING CHAPTER: Nihilism
PATCHING CHAPTER: Pragmatism
PATCHING CHAPTER: Idealism


##### Export each chapter into markdown format and read the contents

In [11]:
# helper function to get the markdown of a job
def get_job_markdown(job_id):
    url = f'{SP_BASE_URL}/long_form/{job_id}/exports/text'
    resp = requests.get(url, auth=API_AUTH)
    download_url = resp.json()['download_url']
    return requests.get(download_url).text

chapter_markdown_contents = []
for chapter_job_id in chapter_ids:
    markdown = get_job_markdown(chapter_job_id)
    chapter_markdown_contents.append(markdown)

##### Create a new knowledge base for the all of the content generated up to this point

In [12]:
book_content_kb = superpowered.create_knowledge_base(
    title='Outlines and Chapters for "Living a Good Life in the Age of AI and Automation"',
)

# create documents from the text generated in the previous steps
for (kb, outline), chapter_markdown in zip(outlines, chapter_markdown_contents):
    # create a document for the outline
    superpowered.create_document_via_text(
        knowledge_base_id=book_content_kb['id'],
        title=f'Chapter Outline for "{kb["title"]}"',
        content=outline,
        auto_context=True,
    )
    # create a document for the chapter
    superpowered.create_document_via_text(
        knowledge_base_id=book_content_kb['id'],
        title=f'Chapter Content for "{kb["title"]}"',
        content=chapter_markdown,
        auto_context=True,
    )


# list documents in the knowledge base
total_docs = len(superpowered.list_documents(book_content_kb['id']))
completed_docs = 0
while completed_docs < total_docs:
    completed_docs = len(superpowered.list_documents(book_content_kb['id'], vectorization_status='COMPLETE'))
    print(f'{completed_docs}/{total_docs} documents completed...')
    time.sleep(5)

0/16 documents completed...
0/16 documents completed...
0/16 documents completed...
7/16 documents completed...
11/16 documents completed...
16/16 documents completed...


##### Run a document review to generate an outline for the introduction and conclusion

In [13]:
active_reading_instructions = f"""\
You are reading through a knowledge base that contains the contents for a book about living a good life in the age of AI and automation.

Each chapter in the book tackles this question from the perspective of a different philosophical school of thought.

Here are the chapters of the book: {', '.join([f'- {kb["title"]}' for kb, _ in outlines])}

Please extract information that is relevant to living a good life in the age of AI and automation. The information that is extracted will be used to write a summary for the book.
"""

introduction_final_review_instructions = f"""\
Your task is to write an outline for an introduction that introduces the book and the general theme of the book. The introduction should be engaging and set the stage for the rest of the book.

Break your introduction into 3-5 sections. Each section should encompass themes from all the chapters in the book rather than focus on a single chapter.

{BOOK_OVERALL_STYLE_GUIDANCE}
"""


conclusion_final_review_instructions = f"""\
Your task is to write an outline for a conclusion that summarizes the book and the general theme of the book. The conclusion should be engaging and leave the reader with a sense of closure.

Break your conclusion into 3-5 sections. Each section should encompass themes from all the chapters in the book rather than focus on a single chapter.

{BOOK_OVERALL_STYLE_GUIDANCE}
"""


# run the document reviews
introduction_outline = run_and_display_review(book_content_kb, active_reading_instructions, introduction_final_review_instructions)
conclusion_outline = run_and_display_review(book_content_kb, active_reading_instructions, conclusion_final_review_instructions)

**Introduction to Living a Good Life in the Age of AI and Automation: A Philosophical Guide**

**Section 1: The Quest for Balance in a Digital World**

In an era where technology, particularly AI and automation, is rapidly reshaping the fabric of our lives, the ancient wisdom of Stoicism, Epicureanism, and Minimalism offers a beacon of light. These philosophies, with their emphasis on virtues, sustainable pleasure, and decluttering both physical and digital spaces, provide a framework for navigating the complexities of modern life. Imagine applying Stoic wisdom to maintain emotional resilience amidst the noise of social media, or embracing Epicurean simplicity in a world of endless digital consumption. Minimalism, with its call to intentional living, challenges us to declutter not just our homes, but our digital footprints as well. Together, these philosophies invite us to seek balance, encouraging a life that harmonizes technology with human well-being.

**Section 2: Navigating Pleasure and Purpose in the Age of Automation**

As AI and automation expand the boundaries of what's possible, they also prompt us to reconsider the nature of pleasure and purpose. Hedonism, with its pursuit of pleasure, and Nihilism, with its challenge to find personal meaning in a seemingly indifferent universe, offer contrasting yet complementary perspectives. Hedonism, in the context of AI, prompts us to explore new forms of pleasure, from immersive virtual realities to the ethical implications of maximizing societal happiness. Nihilism, on the other hand, empowers us to forge our own paths of significance, creating personal values and meanings in a world where traditional benchmarks are constantly evolving. These philosophies encourage us to reflect on what truly matters, guiding us in the pursuit of a fulfilling life amidst technological change.

**Section 3: Ethical and Practical Wisdom for the Future**

Pragmatism and Idealism, each in their own way, address the ethical and practical challenges posed by AI and automation. Pragmatism, with its focus on practical outcomes and adaptability, urges us to consider the broader impacts of technological advancements on society, advocating for ethical decision-making and meaningful online interactions. Idealism, by questioning the nature of reality in the age of AI, invites us to ponder the philosophical implications of consciousness and identity. These perspectives encourage us to think critically about the development and use of technology, ensuring that it serves to enhance rather than diminish the collective well-being.

**Section 4: Embracing Change with Authenticity and Creativity**

Existentialism and the evolving landscape of Minimalism challenge us to live authentically and creatively in a world transformed by AI and automation. Existentialism, emphasizing freedom and self-creation, calls us to actively construct personal meaning in an era of profound change. Meanwhile, Minimalism's principles of simplicity and intentionality guide us in making choices that prioritize humanity, enabling a life that is both intentional and meaningful. Together, these philosophies inspire us to embrace the beauty and chaos of the digital age with open arms, fostering resilience, joy, and a deep sense of purpose.

**Engaging with the Philosophies of a Good Life**

As we journey through the age of AI and automation, the wisdom of these philosophical schools offers invaluable insights for living a good life. By weaving together their teachings, we can navigate the challenges and opportunities of the modern world with confidence, resilience, and a deep commitment to ethical and fulfilling living. This book invites readers to explore these philosophies not as abstract theories, but as practical guides for thriving in an ever-changing landscape, ensuring that technology enhances, rather than detracts from, the human experience.

Active Reading Progress for Outlines and Chapters for "Living a Good Life in the Age of AI and Automation": 100%|██████████| 1/1 [02:09<00:00, 129.89s/it]      


**Embracing Philosophy in the Digital Age: A Guide to Living Well with AI and Automation**

**Introduction to Philosophical Living Amidst Technological Change**
The age of AI and automation brings with it not just technological advancements but profound questions about the nature of a good life. As we navigate this new landscape, ancient and modern philosophies offer valuable insights into living well amidst rapid change. From Stoicism's call for virtue and resilience to Hedonism's pursuit of pleasure, each philosophy provides a unique lens through which to view our digital existence.

**Finding Balance and Meaning in a High-Tech World**
- **Stoicism and Minimalism: The Art of Control and Simplicity**
  Stoicism teaches us the importance of focusing on what we can control and letting go of what we cannot, a principle that becomes increasingly relevant in an era where technology often feels out of our grasp. Similarly, Minimalism encourages us to declutter our digital and physical lives, advocating for a more intentional and focused engagement with technology. Together, these philosophies guide us toward a balanced and meaningful existence, emphasizing quality over quantity in our interactions with the digital world.

- **Hedonism and Epicureanism: Navigating Pleasure and Purpose**
  In the digital age, Hedonism and Epicureanism offer perspectives on finding joy and satisfaction. Hedonism reminds us of the potential for technology to enhance pleasure, from immersive virtual realities to the simple joys of a well-curated playlist. Epicureanism, on the other hand, warns against the empty calories of digital indulgence, advocating for a life of meaningful pleasures and deep connections, both online and off. These philosophies encourage us to seek a harmonious relationship with technology, one that enhances rather than detracts from our well-being.

- **Existentialism and Nihilism: Crafting Personal Meaning**
  Existentialism and Nihilism challenge us to find personal meaning in a world where traditional markers of identity and achievement are rapidly evolving. In the face of AI's impact on work and society, these philosophies empower us to create our own values and definitions of success. They invite us to embrace the freedom and responsibility that come with technological advancements, encouraging us to forge paths of significance that are uniquely our own.

- **Pragmatism and Idealism: Shaping the Future Together**
  Pragmatism and Idealism provide frameworks for engaging with technology in ways that promote human welfare and ethical progress. Pragmatism focuses on practical outcomes and adaptability, urging us to consider the consequences of AI and automation on society. Idealism, meanwhile, challenges us to envision a future where technology uplifts the human spirit, fostering innovation and creativity. Together, these philosophies inspire us to shape a world where technology serves the greater good.

**Conclusion: A Tapestry of Philosophical Wisdom for the Digital Age**
As we stand at the crossroads of human history and technological evolution, the wisdom of Stoicism, Epicureanism, Minimalism, Hedonism, Existentialism, Nihilism, Pragmatism, and Idealism lights the way. Each philosophy, with its unique insights and principles, offers valuable guidance for navigating the complexities of life in the age of AI and automation. By weaving these philosophical threads into the fabric of our daily lives, we can craft a future that is not only technologically advanced but also deeply human. Let us embrace this journey with open minds and hearts, ready to explore the vast potential of living well in the digital age.

Active Reading Progress for Outlines and Chapters for "Living a Good Life in the Age of AI and Automation": 100%|██████████| 1/1 [01:50<00:00, 110.21s/it]      


##### Generate actual introduction and conclusion chapters with long-form

In [14]:
introduction_job_id = submit_long_form_job(book_content_kb, introduction_outline)
conclusion_job_id = submit_long_form_job(book_content_kb, conclusion_outline)

##### Edit titles, export to markdown

In [15]:
patch_chapter_title('Introduction', introduction_job_id)
patch_chapter_title('Conclusion', conclusion_job_id)
introduction_markdown = get_job_markdown(introduction_job_id)
conclusion_markdown = get_job_markdown(conclusion_job_id)

##### Write each markdown file to our book folder and prepend some chapter info

In [16]:
full_book_contents = [introduction_markdown] + chapter_markdown_contents + [conclusion_markdown]

fnames = []
for i, chapter_markdown in enumerate(full_book_contents):
    fname = f'{BOOK_DIRECTORY}/chapter_{i}.md'
    with open(fname, 'w') as f:
        f.write(chapter_markdown)
    fnames.append(fname)

##### Use pandoc to create the book

In [17]:
import subprocess

def generate_epub(output_filename, image_filename):
    # Command parts that are static
    command = [
        'pandoc',
        '-o', output_filename,
        '--toc',
        '--epub-cover-image', image_filename,
        '--metadata', f'title={BOOK_TITLE}',
        '--metadata', f'toc-title={BOOK_TITLE}',
        '--metadata', 'rights=Copyright © 2024 by Superpowered AI. All rights reserved.',
        '--metadata', 'author=Superpowered AI',
    ]

    # Combine the static parts of the command with the filenames
    command.extend(fnames)

    # Execute the command
    try:
        subprocess.run(command, check=True)
        print(f"EPUB successfully created: {output_filename}")
    except subprocess.CalledProcessError as e:
        print(f"Error during EPUB creation: {e}")


generate_epub(f'{BOOK_DIRECTORY}/living_a_good_life.epub', f'{BOOK_DIRECTORY}/living_a_good_life_ai.png')

EPUB successfully created: living_a_good_life/living_a_good_life.epub
